# Some imports 

In [1]:
!pip install tensordict
!pip install torchviz
from torchvision.models import convnext_tiny,ConvNeXt_Tiny_Weights
from torchvision.io import read_image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.5/986.5 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=f409643b869e66203266ce85799f03d2d970df45eca722b7c76d40e648579eab
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [2]:
# Testing convnext for class prediction on imagenet
convnext_weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1
model=convnext_tiny(weights=convnext_weights)
#print(model)
model.eval()

preprocess=convnext_weights.transforms()
img=read_image("/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/test/ILSVRC2012_test_00000046.JPEG")
batch=preprocess(img).unsqueeze(0)

prediction=model(batch).squeeze(0).softmax(0)
class_id= prediction.argmax().item()
score=prediction[class_id].item()
category_name=convnext_weights.meta["categories"][class_id]
print(f"{category_name} : {score*100} % ")

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:00<00:00, 188MB/s]
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


African elephant : 35.99937856197357 % 


## Remove the top layer and Extracting features vectors from the convnext 

In [3]:
import torch
convnext = convnext_tiny(weights=convnext_weights)

convnext_no_fc= torch.nn.Sequential(*list(convnext.children())[:-1]) # strips off last linear layer
#print(convnext_no_fc)
preprocess=convnext_weights.transforms()
img=read_image("/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/test/ILSVRC2012_test_00000046.JPEG")
batch=preprocess(img).unsqueeze(0)

prediction=convnext_no_fc(batch)
# Convert pytorch model to flow of data diagram
from torchviz import make_dot, make_dot_from_trace
model_dia=make_dot(convnext(batch),params=dict(convnext_no_fc.named_parameters()), show_attrs=True, show_saved=True)
model_dia.format="png"
model_dia.render("convnext_model_dia")
print("#####################SHOW THE MODEL NODES##############")
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
train_nodes, eval_nodes = get_graph_node_names(convnext)
for i,j in enumerate(train_nodes):
    print(f"Train node {i} : {j}")
    
print("##############THE NODES TO BE SELECTED FROM THE CONVNEXT MODEL#############################")
list_of_layers_index=[9,48,87,120,153,175,192,225]
layers_to_be_printed=dict()
for i,j in enumerate(train_nodes):
    if i in list_of_layers_index:
        print(f"Train node {i} : {j}")
        layers_to_be_printed[j]=f"Output Layer {i}"
convnext_return_nodes=layers_to_be_printed
convnext_feature_extractor=create_feature_extractor(convnext,return_nodes=convnext_return_nodes)
img=read_image("/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/test/ILSVRC2012_test_00000046.JPEG")
batch=preprocess(img).unsqueeze(0)
feature_list=convnext_feature_extractor(batch)
print("##########################PRINTING THE SHAPES OF THE FEATURES TO BE EXRACTED###############")
for x in feature_list:
    print(torch.flatten(feature_list[x]).shape)

#####################SHOW THE MODEL NODES##############
Train node 0 : x
Train node 1 : features.0
Train node 2 : features.1.0.features_1_0_layer_scale
Train node 3 : features.1.0.block.0
Train node 4 : features.1.0.block.1
Train node 5 : features.1.0.block.2
Train node 6 : features.1.0.block.3
Train node 7 : features.1.0.block.4
Train node 8 : features.1.0.block.5
Train node 9 : features.1.0.block.6
Train node 10 : features.1.0.mul
Train node 11 : features.1.0.stochastic_depth
Train node 12 : features.1.0.add
Train node 13 : features.1.1.features_1_1_layer_scale
Train node 14 : features.1.1.block.0
Train node 15 : features.1.1.block.1
Train node 16 : features.1.1.block.2
Train node 17 : features.1.1.block.3
Train node 18 : features.1.1.block.4
Train node 19 : features.1.1.block.5
Train node 20 : features.1.1.block.6
Train node 21 : features.1.1.mul
Train node 22 : features.1.1.stochastic_depth
Train node 23 : features.1.1.add
Train node 24 : features.1.2.features_1_2_layer_scale
Train

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


## Now features extraction from AlexNet

In [4]:
from torchvision.models import alexnet,AlexNet_Weights
alexnet_weights=AlexNet_Weights.IMAGENET1K_V1
model=alexnet(weights=alexnet_weights)
#print(model)
model.eval()

preprocess=alexnet_weights.transforms()
img=read_image("/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/test/ILSVRC2012_test_00000046.JPEG")
batch=preprocess(img).unsqueeze(0)

prediction=model(batch).squeeze(0).softmax(0)
class_id= prediction.argmax().item()
score=prediction[class_id].item()
category_name=alexnet_weights.meta["categories"][class_id]
print(f"{category_name} : {score*100} % ")

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 193MB/s]


African elephant : 75.67582726478577 % 


In [5]:
import torch
alexnet_model=alexnet(weights=alexnet_weights)

print(alexnet_model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
preprocess=alexnet_weights.transforms()
img=read_image("/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/test/ILSVRC2012_test_00000046.JPEG")
batch=preprocess(img).unsqueeze(0)
print(batch.shape)
prediction=alexnet_model(batch)
from torchviz import make_dot, make_dot_from_trace
model_dia=make_dot(alexnet_model(batch),params=dict(alexnet_model.named_parameters()), show_attrs=True, show_saved=True)

torch.Size([1, 3, 224, 224])


In [7]:
model_dia.format="png"
model_dia.render("alexnet_model_dia")

'alexnet_model_dia.png'

In [8]:
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
train_nodes, eval_nodes = get_graph_node_names(alexnet_model)

In [9]:
list_of_layers_index=[1,4,7,9,11,17,20,22]

In [10]:
for i,j in enumerate(train_nodes):
    print(f"Train node {i} : {j}")

Train node 0 : x
Train node 1 : features.0
Train node 2 : features.1
Train node 3 : features.2
Train node 4 : features.3
Train node 5 : features.4
Train node 6 : features.5
Train node 7 : features.6
Train node 8 : features.7
Train node 9 : features.8
Train node 10 : features.9
Train node 11 : features.10
Train node 12 : features.11
Train node 13 : features.12
Train node 14 : avgpool
Train node 15 : flatten
Train node 16 : classifier.0
Train node 17 : classifier.1
Train node 18 : classifier.2
Train node 19 : classifier.3
Train node 20 : classifier.4
Train node 21 : classifier.5
Train node 22 : classifier.6


In [11]:
layers_to_be_printed=dict()
for i,j in enumerate(train_nodes):
    if i in list_of_layers_index:
        print(f"Train node {i} : {j}")
        layers_to_be_printed[j]=f"Output Layer {i}"
print(layers_to_be_printed)

Train node 1 : features.0
Train node 4 : features.3
Train node 7 : features.6
Train node 9 : features.8
Train node 11 : features.10
Train node 17 : classifier.1
Train node 20 : classifier.4
Train node 22 : classifier.6
{'features.0': 'Output Layer 1', 'features.3': 'Output Layer 4', 'features.6': 'Output Layer 7', 'features.8': 'Output Layer 9', 'features.10': 'Output Layer 11', 'classifier.1': 'Output Layer 17', 'classifier.4': 'Output Layer 20', 'classifier.6': 'Output Layer 22'}


In [12]:
alexnet_return_nodes=layers_to_be_printed

alexnet_feature_extractor=create_feature_extractor(alexnet_model,return_nodes=alexnet_return_nodes)
img=read_image("/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/test/ILSVRC2012_test_00000046.JPEG")
batch=preprocess(img).unsqueeze(0)
feature_list=alexnet_feature_extractor(batch)

for x in feature_list:
    print(torch.flatten(feature_list[x]).shape)

torch.Size([193600])
torch.Size([139968])
torch.Size([64896])
torch.Size([43264])
torch.Size([43264])
torch.Size([4096])
torch.Size([4096])
torch.Size([1000])


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


### Now looping through all images used in the seen vs imagined study to regenrate the features and save them as files for decoding

In [13]:
# from PIL import Image
# import requests
# import torchvision.transforms.functional as TF
# count=0
# import pandas as pd 
# df_url=pd.read_csv("/kaggle/input/seen-vs-imagined-dataset-project/imageNet_URLTraining_fixed.csv",header=None).astype("str").values.tolist()
# imagenet_train_loc="/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/"
# for i,j in enumerate(df_url):
    
   
#     try:
#         image_file_path=imagenet_train_loc+f"n0{j}/"+df_url[i][2]
#         url = df_url[i][0]
#         #print(url)
#         try:
#             img = Image.open(requests.get(url, stream=True).raw)
#             print(f"URL ALL GOOD for : count:{i} file_ame:{df_url[i][1]}")
#         except:
#             print(f"URL:{df_url[i][0]} not opening trying for imagenet")
#             try:
#                 img=Image.open(image_file_path)
#                 print("file opened from imagenet")
#             except:
#                 print(f"image {df_url[i][1]} found online or in imagenet")
#         count+=1
#     except:
#         print(f"{df_url[i][1]} file not found")
#     print(img.size)
# print(f"correctly loaded files are {count}/{len(df_url)}")
#         #print("ERROR at finding file name:",count)
    

# code for data loader

In [14]:
import os
import numpy as np
from torch.utils.data import DataLoader
from PIL import Image
from torch.utils.data import Dataset
from pathlib import Path
import torchvision.io
class IMG_Dataset(Dataset):
    def __init__(self, image_dir, transform = None):
        self.image_dir = image_dir
        
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        
        image = read_image(img_path,mode=torchvision.io.ImageReadMode.RGB)


        if self.transform is  not None:
            try:
                transformed_img= self.transform(image)
                image = transformed_img
            except:
                print(f"error found at file {img_path}")
                print(image.shape)

        return image,img_path

In [15]:
def get_loaders(
        train_dir,
        val_dir,
        batch_size,
        train_transform,
        test_transform,
        num_workers=4,
        pin_memory=True
):
    train_dataset = IMG_Dataset(
        image_dir=train_dir,
        transform=train_transform
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True
    )

    test_dataset = IMG_Dataset(
        image_dir=val_dir,
        transform=test_transform
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True
    )

    return train_loader, test_loader


# Alexnet features 

# for training data imgs 

In [16]:
alex_train_transform =alexnet_weights.transforms()
alex_test_transforms =alexnet_weights.transforms()
train_dir="/kaggle/input/seen-vs-imagined-horikawa-et-al-imagenet-subset/images/training"
test_dir="/kaggle/input/seen-vs-imagined-horikawa-et-al-imagenet-subset/images/test"
num_workers=2
pin_memory=True
batch_size=1
device="cpu"
train_loader, test_loader = get_loaders(
        train_dir,
        test_dir,
        batch_size,
        alex_train_transform,
        alex_test_transforms,
        num_workers,
        pin_memory,
    )

In [17]:
from tqdm import tqdm
import torch
import torch.nn as nn

from tensordict import TensorDict
from tensordict.nn import TensorDictModule, TensorDictSequential
alexnet_feature_dict=dict()
loop = tqdm(train_loader)
for batch_idx,(data,image_path)  in enumerate(loop):
    
    data = data.to(device=device)
    feature_list=alexnet_feature_extractor(data)
    flattened_feature_dict=dict()
    for x in feature_list:
        #print(feature_list[x].shape)
        y=torch.flatten(feature_list[x]).detach().cpu().numpy()#flatten in code not working
        sampled_size=1000
        sampled_features=np.random.choice(y,size=(1000),replace=False)
        flattened_feature_dict[x]=torch.unsqueeze(torch.from_numpy(sampled_features),0)
        #print(flattened_feature_dict[x].shape)
        
    
    img_name=str(*image_path).split("/")[-1][1:-5]
    alexnet_feature_dict[img_name]=flattened_feature_dict

#     for x in feature_list:
#         print(torch.flatten(feature_list[x]).shape)
#print(alexnet_feature_dict)
alex_train_fatures_dict_torch=TensorDict(alexnet_feature_dict,batch_size=1)
torch.save(alex_train_fatures_dict_torch,"alexnet_dict_train.pt")    

100%|██████████| 1200/1200 [01:11<00:00, 16.76it/s]


In [18]:

y =alex_train_fatures_dict_torch["01518878_10042"]["Output Layer 11"].detach().cpu().numpy()
print(y.shape)


(1, 1000)


# for test data 

In [19]:
from tqdm import tqdm
import torch
import torch.nn as nn

from tensordict import TensorDict
from tensordict.nn import TensorDictModule, TensorDictSequential
alexnet_feature_dict_test=dict()
loop = tqdm(test_loader)
for batch_idx,(data,image_path)  in enumerate(loop):
    #print("new img features")
    data = data.to(device=device)
    feature_list=alexnet_feature_extractor(data)
    flattened_feature_dict=dict()
    for x in feature_list:
        #print(feature_list[x].shape)
        
        y=torch.flatten(feature_list[x]).detach().cpu().numpy()#flatten in code not working
        sampled_size=1000
        sampled_features=np.random.choice(y,size=(1000),replace=False)
        flattened_feature_dict[x]=torch.unsqueeze(torch.from_numpy(sampled_features),0)
        #print(flattened_feature_dict[x].shape)
    img_name=str(*image_path).split("/")[-1][1:-5]
    
    alexnet_feature_dict_test[img_name]=flattened_feature_dict

#     for x in feature_list:
#         print(torch.flatten(feature_list[x]).shape)

alex_train_fatures_dict_torch_test=TensorDict(alexnet_feature_dict_test,batch_size=1)
torch.save(alex_train_fatures_dict_torch_test,"alexnet_dict_test.pt")    

100%|██████████| 50/50 [00:03<00:00, 16.18it/s]


In [20]:
# print(alex_train_fatures_dict_torch_test)
# x =alex_train_fatures_dict_torch_test["01443537_22563"]["Output Layer 11"].detach().cpu().numpy()
# print(x.shape)

# For convnext 

## training data 

In [21]:
import os
os.makedirs("/kaggle/working/convnext_train", exist_ok=True)

In [22]:
from tqdm import tqdm
import torch
import torch.nn as nn

from tensordict import TensorDict
from tensordict.nn import TensorDictModule, TensorDictSequential
convnext_feature_dict_train=dict()
loop = tqdm(train_loader)
for batch_idx,(data,image_path)  in enumerate(loop):       
    #print("new img features")
    data = data.to(device=device)
    feature_list=convnext_feature_extractor(data)
    flattened_feature_dict=dict()
    for x in feature_list:
        #print(feature_list[x].shape)
        
        y=torch.flatten(feature_list[x]).detach().cpu().numpy()#flatten in code not working
        sampled_size=1000
        sampled_features=np.random.choice(y,size=(1000),replace=False)
        flattened_feature_dict[x]=torch.unsqueeze(torch.from_numpy(sampled_features),0)
        #print(flattened_feature_dict[x].shape)
    img_name=str(*image_path).split("/")[-1][1:-5]
    
    convnext_feature_dict_train[img_name]=flattened_feature_dict
#     for x in feature_list:
#         print(torch.flatten(feature_list[x]).shape)

convnext_train_fatures_dict_torch_train=TensorDict(convnext_feature_dict_train,batch_size=1)
torch.save(convnext_train_fatures_dict_torch_train,"convnext_features_train.pt")  

100%|██████████| 1200/1200 [03:05<00:00,  6.48it/s]


In [23]:
#!zip -r "conv_next_train.zip" "/kaggle/working/convnext_train/"

# convnext test img features 

In [24]:
from tqdm import tqdm
import torch
import torch.nn as nn

from tensordict import TensorDict
from tensordict.nn import TensorDictModule, TensorDictSequential
convnext_feature_dict_test=dict()
loop = tqdm(test_loader)
for batch_idx,(data,image_path)  in enumerate(loop):
    
    data = data.to(device=device)
    feature_list=convnext_feature_extractor(data)
    flattened_feature_dict=dict()
    for x in feature_list:
        #print(feature_list[x].shape)
        
        y=torch.flatten(feature_list[x]).detach().cpu().numpy()#flatten in code not working
        sampled_size=1000
        sampled_features=np.random.choice(y,size=(1000),replace=False)
        flattened_feature_dict[x]=torch.unsqueeze(torch.from_numpy(sampled_features),0)
        #print(flattened_feature_dict[x].shape)
    img_name=str(*image_path).split("/")[-1][1:-5]
    convnext_feature_dict_test[img_name]=flattened_feature_dict
#     for x in feature_list:
#         print(torch.flatten(feature_list[x]).shape)

convnext_train_fatures_dict_torch_test=TensorDict(convnext_feature_dict_test,batch_size=1)
torch.save(convnext_train_fatures_dict_torch_test,"convnext_features_test.pt")  

100%|██████████| 50/50 [00:08<00:00,  5.93it/s]


In [25]:
!zip -r "test_train_img_features.zip" "/kaggle/working/alexnet_dict_test.pt" "/kaggle/working/alexnet_dict_train.pt" "/kaggle/working/convnext_features_test.pt" "/kaggle/working/convnext_features_train.pt"

  adding: kaggle/working/alexnet_dict_test.pt (deflated 65%)
  adding: kaggle/working/alexnet_dict_train.pt (deflated 65%)
  adding: kaggle/working/convnext_features_test.pt (deflated 11%)
  adding: kaggle/working/convnext_features_train.pt (deflated 12%)
